# Data Cleaning
- only keeping movies that were released (dropping anything pre-release)
- get rid of the columns that we wont use
- create binary response 

In [19]:
import pandas as pd
import numpy as np

In [61]:
# importing csv file
movies_metadata = pd.read_csv('movies_metadata.csv')

C:\Users\csky2\AppData\Local\Temp\ipykernel_4812\1536947796.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('movies_metadata.csv')


In [62]:
# only keeping movies that were released (dropping anything pre-release)
# drops about 400 movies
movies_metadata = movies_metadata.drop(np.where(movies_metadata['status'] != 'Released')[0])

In [63]:
# dropping 9 columns that we will not use
movies_metadata = movies_metadata.drop(['adult', 'homepage', 'id', 'imdb_id', 'original_title', 'poster_path', 'status', 'video', 'overview'], axis=1)

In [64]:
# cleaning belongs_to_collection - making it binary
movies_metadata.loc[~movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 1  # not nan
movies_metadata.loc[movies_metadata.belongs_to_collection.isnull(), 'belongs_to_collection'] = 0

In [65]:
movies_metadata

,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
0,1,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",NaN,Toy Story,7.7,5415.0
1,0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,1,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,1,0,"[{'id': 35, 'name': 'Comedy'}]",en,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,0.072051,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Rising and falling between a man and woman,Subdue,4.0,1.0
45462,0,0,"[{'id': 18, 'name': 'Drama'}]",tl,0.178241,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",NaN,Century of Birthing,9.0,3.0
45463,0,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,0.903007,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",A deadly game of wits.,Betrayal,3.8,6.0
45464,0,0,[],en,0.003503,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,87.0,[],NaN,Satan Triumphant,0.0,0.0


In [18]:
movies_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [26]:
movies_metadata.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [27]:
movies_metadata.revenue.describe()

count    4.546000e+04
mean     1.120935e+07
std      6.433225e+07
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.787965e+09
Name: revenue, dtype: float64

In [31]:
movies_metadata[['revenue']].value_counts()

revenue     
0.000000e+00    38052
1.200000e+07       20
1.100000e+07       19
1.000000e+07       19
2.000000e+06       18
                ...  
6.101904e+06        1
6.101046e+06        1
6.100000e+06        1
6.086508e+06        1
2.787965e+09        1
Length: 6863, dtype: int64

In [32]:
movies_metadata.shape

(45466, 24)

In [34]:
movies_metadata.runtime.describe()

count    45203.000000
mean        94.128199
std         38.407810
min          0.000000
25%         85.000000
50%         95.000000
75%        107.000000
max       1256.000000
Name: runtime, dtype: float64